In [20]:
from selenium import webdriver
from pyshadow.main import Shadow
from selenium.webdriver.common.keys import Keys
import re
import time
import string

FILE = "american-english.txt"
ALLOWABLE_CHARACTERS = set(string.ascii_letters)
ALLOWED_ATTEMPTS = 6
WORD_LENGTH = 5

In [21]:
wordlist = [word.strip() for word in open(FILE, "r").readlines()]

#creating a set of valid words according to the rules of Wordle
WORDS = {
    word.lower()
    for word in wordlist    
    if len(word) == WORD_LENGTH and set(word) < ALLOWABLE_CHARACTERS
    }
WORDS

{'essay',
 'pried',
 'alive',
 'manly',
 'tipsy',
 'crest',
 'pivot',
 'furry',
 'occur',
 'manor',
 'piggy',
 'spill',
 'weigh',
 'diver',
 'vouch',
 'sever',
 'fifty',
 'fritz',
 'smote',
 'rerun',
 'chess',
 'outdo',
 'crass',
 'shore',
 'agape',
 'bread',
 'knoll',
 'hotly',
 'media',
 'brain',
 'raise',
 'gusto',
 'broom',
 'flesh',
 'straw',
 'skill',
 'chide',
 'golem',
 'scrum',
 'fanny',
 'drawn',
 'sieve',
 'silly',
 'mange',
 'vigor',
 'radii',
 'blind',
 'kneel',
 'allot',
 'boule',
 'reach',
 'bushy',
 'eager',
 'femme',
 'eerie',
 'awake',
 'ember',
 'adapt',
 'inept',
 'impel',
 'bongo',
 'cross',
 'icily',
 'skunk',
 'quasi',
 'bigot',
 'palsy',
 'foyer',
 'mirth',
 'blare',
 'omega',
 'limit',
 'debar',
 'itchy',
 'sweet',
 'tatty',
 'tract',
 'edify',
 'throb',
 'umbra',
 'hoist',
 'creed',
 'delta',
 'extol',
 'spied',
 'ennui',
 'amble',
 'revue',
 'stock',
 'roast',
 'stung',
 'drive',
 'worth',
 'jewel',
 'carve',
 'unzip',
 'chaos',
 'point',
 'sappy',
 'bunch',


In [22]:
from collections import Counter
from itertools import chain

#Creating a Counter Object that counts the number of occurences
#of each letter used across all valid Wordle words

LETTER_COUNTER = Counter(chain.from_iterable(WORDS))

LETTER_COUNTER


Counter({'e': 1233,
         's': 669,
         'a': 979,
         'y': 425,
         'p': 367,
         'r': 899,
         'i': 671,
         'd': 393,
         'l': 719,
         'v': 153,
         'm': 316,
         'n': 575,
         't': 729,
         'c': 477,
         'o': 754,
         'f': 230,
         'u': 467,
         'g': 311,
         'w': 195,
         'h': 389,
         'z': 40,
         'b': 281,
         'k': 210,
         'q': 29,
         'x': 37,
         'j': 27})

In [23]:
#finding the total occurances of all letters
total = len(WORDS)*WORD_LENGTH
LETTER_FREQUENCY = {character: value / total 
                    for character, value in LETTER_COUNTER.items()}

LETTER_FREQUENCY

{'e': 0.10652267818574514,
 's': 0.057796976241900645,
 'a': 0.08457883369330453,
 'y': 0.0367170626349892,
 'p': 0.031706263498920084,
 'r': 0.07766738660907127,
 'i': 0.057969762419006476,
 'd': 0.033952483801295896,
 'l': 0.06211663066954644,
 'v': 0.013218142548596112,
 'm': 0.027300215982721383,
 'n': 0.04967602591792657,
 't': 0.06298056155507559,
 'c': 0.04120950323974082,
 'o': 0.0651403887688985,
 'f': 0.019870410367170625,
 'u': 0.04034557235421166,
 'g': 0.026868250539956805,
 'w': 0.016846652267818573,
 'h': 0.033606911447084234,
 'z': 0.0034557235421166306,
 'b': 0.02427645788336933,
 'k': 0.01814254859611231,
 'q': 0.002505399568034557,
 'x': 0.0031965442764578834,
 'j': 0.002332613390928726}

In [24]:
def calculate_word_commonality(word):
    score = 0
    for char in word:
        score += LETTER_FREQUENCY[char]
    return score / (WORD_LENGTH - len(set(word)) + 1)

In [25]:
import operator

def sort_by_word_commonality(words):
    sort_by = operator.itemgetter(1)
    return sorted(
        [(word, calculate_word_commonality(word)) for word in words],
        key = sort_by,
        reverse=True,
    )

def display_word_table(word_commonalities):
    for (word,freq) in word_commonalities:
        print(f"{word:<10} | {freq:<5.2}")


In [26]:
def input_word():
    while True:
        word = input("Input the word you entered> ")
        if len(word) == WORD_LENGTH and word.lower() in WORDS:
            break
    return word.lower()

def input_response():
    print("Type the color-coded reply from Wordle:")
    print(" G for Green")
    print(" Y for Yellow")
    print(" ? for Gray")
    while True:
        response = input("Response from Wordle> ")
        if len(response) == WORD_LENGTH and set(response) <= {"G", "Y", "?"}:
            break
        else:
            print(f"Error - invalid answer {response}")
    return response

In [27]:
def match_word_vector(word, word_vector):
    assert len(word) == len(word_vector)
    for letter, v_letter in zip(word, word_vector):
        if letter not in v_letter:
            return False
    return True

def match(word_vector, possible_words):
    return [word for word in possible_words if match_word_vector(word, word_vector)]

In [28]:
def enter_word(browser, word):
    shadow = Shadow(browser)
    wordleBot = browser.find_element_by_tag_name('html')
    wordleBot.click()
    time.sleep(1)
    wordleBot.send_keys(word)
    wordleBot.send_keys(Keys.ENTER)
    time.sleep(1)
    finalEvaluation = list(word)
    gameRow = shadow.find_element("game-row[letters="+word+"]")
    for char in set(word):
        gameTile = shadow.find_elements(gameRow,"game-tile[letter=" + char +"]")
        index = -1
        for elem in gameTile:
            if elem.get_attribute("evaluation") == 'present':
                finalEvaluation[word.index(char, index + 1, len(word))] = 'Y' 
            if elem.get_attribute("evaluation") == 'correct':
                finalEvaluation[word.index(char, index + 1, len(word))] = 'G'
            if elem.get_attribute("evaluation") == 'absent':
                finalEvaluation[word.index(char, index + 1, len(word))] = '?'
            index = word.index(char, index + 1, len(word))
    return finalEvaluation

In [29]:
def solve():
    browser = webdriver.Chrome(executable_path ='/usr/local/bin/chromedriver')
    shadow = Shadow(browser)
    browser.get('http://www.powerlanguage.co.uk/wordle/')
    time.sleep(1)
    possible_words = WORDS.copy()
    word_vector = [set(string.ascii_lowercase) for _ in range(WORD_LENGTH)]
    yLetters = set()
    for attempt in range(1, ALLOWED_ATTEMPTS + 1):
        print(f"Attempt {attempt} with {len(possible_words)} possible words")
        sortedWords = sort_by_word_commonality(possible_words)
        display_word_table(sortedWords[:15])
        i = 0
        word = sortedWords[0][0]
        while yLetters.issubset(set(word)) == False:
            i = i + 1
            word = sortedWords[i][0]
        response = enter_word(browser,word)
        for idx, letter in enumerate(response):
            if letter == "G":
                word_vector[idx] = {word[idx]}
            elif letter == "Y":
                try:
                    word_vector[idx].remove(word[idx])
                    yLetters.add(word[idx])
                except KeyError:
                    pass
            elif letter == "?":
                for vector in word_vector:
                    try:
                        vector.remove(word[idx])
                    except KeyError:
                        pass
        possible_words = match(word_vector, possible_words)
    

In [39]:
def solve():
    browser = webdriver.Chrome(executable_path ='/usr/local/bin/chromedriver')
    shadow = Shadow(browser)
    browser.get('https://wordle.berknation.com/')
    time.sleep(10)
    possible_words = WORDS.copy()
    word_vector = [set(string.ascii_lowercase) for _ in range(WORD_LENGTH)]
    yLetters = set()
    for attempt in range(1, ALLOWED_ATTEMPTS + 1):
        print(f"Attempt {attempt} with {len(possible_words)} possible words")
        sortedWords = sort_by_word_commonality(possible_words)
        display_word_table(sortedWords[:15])
        i = 0
        word = sortedWords[0][0]
        while yLetters.issubset(set(word)) == False:
            i = i + 1
            word = sortedWords[i][0]
        response = enter_word(browser,word)
        for idx, letter in enumerate(response):
            if letter == "G":
                word_vector[idx] = {word[idx]}
            elif letter == "Y":
                try:
                    word_vector[idx].remove(word[idx])
                    yLetters.add(word[idx])
                except KeyError:
                    pass
            elif letter == "?":
                for vector in word_vector:
                    try:
                        vector.remove(word[idx])
                    except KeyError:
                        pass
        possible_words = match(word_vector, possible_words)
    

In [40]:
solve()

<ipython-input-39-4da44e9dbd5c>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path ='/usr/local/bin/chromedriver')


Attempt 1 with 2315 possible words
later      | 0.39 
alter      | 0.39 
alert      | 0.39 
arose      | 0.39 
irate      | 0.39 
stare      | 0.39 
raise      | 0.38 
arise      | 0.38 
renal      | 0.38 
learn      | 0.38 
snare      | 0.38 
saner      | 0.38 
steal      | 0.37 
least      | 0.37 
slate      | 0.37 


<ipython-input-28-51aad33ca172>:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  wordleBot = browser.find_element_by_tag_name('html')


QA--QAQA True
Attempt 2 with 40 possible words
nosey      | 0.32 
honey      | 0.29 
shied      | 0.29 
sinew      | 0.29 
spied      | 0.29 
money      | 0.29 
piney      | 0.28 
boney      | 0.28 
video      | 0.28 
dicey      | 0.28 
coven      | 0.28 
dopey      | 0.27 
women      | 0.27 
widen      | 0.26 
bused      | 0.26 
QA--QAQA True
Attempt 3 with 6 possible words
video      | 0.28 
bicep      | 0.26 
chief      | 0.26 
cheek      | 0.15 
embed      | 0.15 
emcee      | 0.13 
QA--QAQA True
Attempt 4 with 1 possible words
video      | 0.28 
QA--QAQA True
Attempt 5 with 1 possible words
video      | 0.28 
QA--QAQA True
Attempt 6 with 1 possible words
video      | 0.28 
QA--QAQA True
